<a href="https://colab.research.google.com/github/Jaksta1/Monte-Carlo-method-for-option-prices/blob/main/skrypt%20mgr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from qmcpy import Sobol

# ---------------------------
# QMC -> normals (QMCPy Sobol, randomize=True gives full Owen scramble)
# ---------------------------
def qmc_normals_qmcpy(n_paths, total_dim, seed=None):
    """Zwraca tablicę (n_paths, total_dim) z punktami Sobola przekształconymi do N(0,1)."""
    sob = Sobol(dimension=total_dim, randomize=True, seed=seed)
    u = sob.gen_samples(n_paths)  # shape (n_paths, total_dim)
    eps = 1e-16
    u = np.clip(u, eps, 1 - eps)
    return norm.ppf(u)


# ---------------------------
# Rozszerzony MARS (LDA + GCV)
# ---------------------------
class ExtendedMARS:
    def __init__(self, max_terms=20, min_leaf=10, penalty=3.0):
        self.max_terms = int(max_terms)
        self.min_leaf = int(min_leaf)
        self.penalty = float(penalty)
        self.basis = []   # list of (a, knot, sign)
        self.coefs = None
        self.intercept_ = 0.0

    def _project(self, X, a):
        return X.dot(a)

    def _gcv(self, y, y_pred, n_params):
        n = len(y)
        rss = np.sum((y - y_pred) ** 2)
        C = n_params + self.penalty
        if n <= C:
            return rss  # fallback
        denom = (1.0 - C / n) ** 2
        return rss / denom

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y).ravel()
        n, d = X.shape
        design = np.ones((n, 1))  # intercept
        self.basis = []

        for step in range(self.max_terms):
            coef_curr, *_ = np.linalg.lstsq(design, y, rcond=None)
            y_pred_curr = design.dot(coef_curr)
            residual = y - y_pred_curr

            # candidate directions: LDA on sign of residuals (two classes)
            try:
                labels = (residual > np.median(residual)).astype(int)
                lda = LinearDiscriminantAnalysis(n_components=1)
                lda.fit(X, labels)
                a_candidate = lda.coef_[0]
                if np.linalg.norm(a_candidate) > 0:
                    a_candidate = a_candidate / (np.linalg.norm(a_candidate) + 1e-12)
                    candidate_dirs = [a_candidate]
                else:
                    candidate_dirs = [np.eye(d)[j] for j in range(d)]
            except Exception:
                candidate_dirs = [np.eye(d)[j] for j in range(d)]

            best_gcv = np.inf
            best_tuple = None  # (a, knot, sign, col)

            for a in candidate_dirs:
                proj = self._project(X, a)
                # candidate knots: percentyle w zakresie [10,90]
                nknots = max(2, min(12, n // max(1, self.min_leaf)))
                percentiles = np.linspace(10, 90, nknots)
                knots = np.unique(np.percentile(proj, percentiles))
                for knot in knots:
                    for sign in (+1, -1):
                        if sign == 1:
                            col = np.maximum(proj - knot, 0.0)[:, None]
                        else:
                            col = np.maximum(knot - proj, 0.0)[:, None]
                        if np.allclose(col, 0.0):
                            continue
                        D = np.hstack([design, col])
                        coef_try, *_ = np.linalg.lstsq(D, y, rcond=None)
                        y_try = D.dot(coef_try)
                        gcv = self._gcv(y, y_try, D.shape[1])
                        if gcv < best_gcv - 1e-12:
                            best_gcv = gcv
                            best_tuple = (a.copy(), float(knot), int(sign), col)

            if best_tuple is None:
                break
            # accept basis
            a_best, knot_best, sign_best, col_best = best_tuple
            self.basis.append((a_best, knot_best, sign_best))
            design = np.hstack([design, col_best])

        # final fit
        coef_final, *_ = np.linalg.lstsq(design, y, rcond=None)
        self.intercept_ = float(coef_final[0])
        if design.shape[1] > 1:
            self.coefs = coef_final[1:].astype(float)
        else:
            self.coefs = np.array([], dtype=float)

        # backward pruning based on GCV (try removing each basis if improves GCV)
        improved = True
        while improved and len(self.basis) > 0:
            improved = False
            # build design once per iteration
            D_full = np.ones((n, 1))
            for (a, knot, sign) in self.basis:
                proj = self._project(X, a)
                col = (np.maximum(proj - knot, 0.0) if sign == 1 else np.maximum(knot - proj, 0.0))[:, None]
                D_full = np.hstack([D_full, col])
            coef_full, *_ = np.linalg.lstsq(D_full, y, rcond=None)
            gcv_full = self._gcv(y, D_full.dot(coef_full), D_full.shape[1])
            # test removals
            for idx in range(len(self.basis)):
                basis_try = self.basis[:idx] + self.basis[idx + 1:]
                D_try = np.ones((n, 1))
                for (a, knot, sign) in basis_try:
                    proj = self._project(X, a)
                    col = (np.maximum(proj - knot, 0.0) if sign == 1 else np.maximum(knot - proj, 0.0))[:, None]
                    D_try = np.hstack([D_try, col])
                coef_try, *_ = np.linalg.lstsq(D_try, y, rcond=None)
                gcv_try = self._gcv(y, D_try.dot(coef_try), D_try.shape[1])
                if gcv_try < gcv_full - 1e-12:
                    # accept pruning
                    self.basis = basis_try
                    self.intercept_ = float(coef_try[0])
                    self.coefs = (coef_try[1:].astype(float) if D_try.shape[1] > 1 else np.array([], dtype=float))
                    improved = True
                    break
        return self

    def predict(self, X):
        X = np.asarray(X)
        n = X.shape[0]
        y_pred = np.full(n, self.intercept_, dtype=float)
        for idx, (a, knot, sign) in enumerate(self.basis):
            proj = self._project(X, a)
            term = (np.maximum(proj - knot, 0.0) if sign == 1 else np.maximum(knot - proj, 0.0))
            coef = self.coefs[idx] if idx < len(self.coefs) else 0.0
            y_pred += coef * term
        return y_pred


# ---------------------------
# LSM + faza II (L_hat, U_hat) używając ExtendedMARS
# ---------------------------
class LSM_MARS_QMC:
    def __init__(self, simulator_func, payoff_func, times, r, dim, mars_params=None, rng_seed=None):
        """
        simulator_func: callable(normals) -> paths shape (n_paths, n_steps, dim)
          where normals shape is (n_paths, n_steps, dim) standard normals.
        payoff_func: callable(t_index, x_vector) -> scalar payoff
        times: array of time grid (length n_steps)
        r: interest rate (annual)
        dim: dimension of underlying (liczba aktywów)
        mars_params: dict for ExtendedMARS
        rng_seed: optional seed for reproducibility
        """
        self.simulator = simulator_func
        self.payoff = payoff_func
        self.times = np.asarray(times)
        self.dt = np.diff(self.times)
        self.r = float(r)
        self.dim = int(dim)
        self.mars_params = mars_params or {}
        self.rng = np.random.default_rng(rng_seed)
        # model parameters (mu, sigma) needed for simulator_one_step in phase II
        self.mu = None
        self.sigma = None

    def set_model_params(self, mu, sigma):
        self.mu = np.asarray(mu, dtype=float)
        self.sigma = np.asarray(sigma, dtype=float)

    def simulator_one_step(self, x_prev, z_stdnormal, step_index):
        """
        Jednokrokowa symulacja GBM (wektorowa). Zakładamy model GBM dla każdego aktywa niezależnie.
        x_prev: (dim,)
        z_stdnormal: (dim,) standard normals
        step_index: indeks kroku docelowego (j), symulujemy z t_{j-1} -> t_j
        """
        if self.mu is None or self.sigma is None:
            raise RuntimeError("set_model_params(mu, sigma) must be called before simulator_one_step.")
        # dt for step from t_{j-1} to t_j
        if step_index <= 0:
            raise ValueError("step_index must be >= 1 for one-step simulation")
        dt = self.times[step_index] - self.times[step_index - 1]
        drift = (self.mu - 0.5 * (self.sigma ** 2)) * dt
        diffusion = self.sigma * np.sqrt(dt) * z_stdnormal
        return x_prev * np.exp(drift + diffusion)

    def _generate_normals(self, n_paths, seed_offset=0):
        n_steps = len(self.times)
        total_dim = n_steps * self.dim
        # seed_offset passed into seed for QMCPy for reproducibility; QMCPy expects integer seed or None
        seed = None if seed_offset is None else int(seed_offset)
        u = qmc_normals_qmcpy(n_paths, total_dim, seed=seed)
        normals = u.reshape(n_paths, n_steps, self.dim)
        return normals

    def price(self, N1=1024, N2=512, max_terms=12, inner_MC=64):
        """
        Wykonaj fazę I (N1) oraz fazę II (N2). Zwróć L_hat i U_hat.
        inner_MC: liczba symulacji wewnętrznych do estymacji E[h_t | X_{t-1}]
        """
        n_steps = len(self.times)
        # Phase I: fit models on N1 QMC paths
        normals1 = self._generate_normals(N1, seed_offset=11)
        X1 = self.simulator(normals1)  # (N1, n_steps, dim)
        # compute immediate payoffs matrix
        B = np.zeros((N1, n_steps))
        for i in range(N1):
            for j in range(n_steps):
                B[i, j] = float(self.payoff(j, X1[i, j, :]))

        models = [None] * n_steps
        # terminal
        models[-1] = None  # terminal value is payoff

        # backward induction LSM + MARS (fit B_j)
        for j in range(n_steps - 2, -1, -1):
            disc = np.exp(-self.r * self.dt[j])
            # target: value = max(immediate payoff, discounted next)
            Y = np.maximum(B[:, j], disc * B[:, j + 1])
            X_train = X1[:, j, :]
            model = ExtendedMARS(max_terms=max_terms, **self.mars_params)
            model.fit(X_train, Y)
            models[j] = model
            cont = model.predict(X_train)
            exercise = B[:, j] > cont
            B[:, j] = np.where(exercise, B[:, j], disc * B[:, j + 1])

        # Phase II: estimate L_hat and U_hat on independent N2 QMC paths
        normals2 = self._generate_normals(N2, seed_offset=999)
        X2 = self.simulator(normals2)
        L_vals = np.zeros(N2)
        U_vals = np.zeros(N2)

        for i in range(N2):
            pi = 0.0
            stopped = False
            for j in range(n_steps):
                t = self.times[j]
                g = float(self.payoff(j, X2[i, j, :]))
                h = float(models[j].predict(X2[i, j, :].reshape(1, -1))[0]) if models[j] else g

                if j > 0:
                    x_prev = X2[i, j - 1, :]
                    zs = self.rng.standard_normal(size=(inner_MC, self.dim))
                    x_next_samples = np.array([self.simulator_one_step(x_prev, zs[k], j) for k in range(inner_MC)])
                    h_vals = models[j].predict(x_next_samples) if models[j] else np.array([float(self.payoff(j, xn)) for xn in x_next_samples])
                    h_cond = float(np.mean(h_vals))
                else:
                    h_cond = h  # zamiast 0

                pi += h - h_cond  # bez dodatkowego dyskontowania

                # Lower bound
                if not stopped and g > h:
                    L_vals[i] = np.exp(-self.r * t) * g
                    stopped = True

                # Upper bound
                cand = np.exp(-self.r * t) * (g - pi)
                if cand > U_vals[i]:
                    U_vals[i] = cand

            if not stopped:
                L_vals[i] = np.exp(-self.r * self.times[-1]) * float(self.payoff(n_steps - 1, X2[i, -1, :]))

        L_hat = float(np.mean(L_vals))
        U_hat = float(np.mean(U_vals))
        return {"L_hat": L_hat, "U_hat": U_hat, "models": models}


# ---------------------------
# GBM simulator (użyty jako przykład)
# ---------------------------
def gbm_simulator_factory(S0, mu, sigma, times):
    """
    simulator(normals) -> paths with shape (n_paths, n_steps, dim)
    normals expected shape: (n_paths, n_steps, dim) standard normals
    """
    S0 = np.array(S0, dtype=float)
    mu = np.array(mu, dtype=float)
    sigma = np.array(sigma, dtype=float)
    times = np.array(times, dtype=float)
    dt = np.diff(times)
    n_steps = len(times)
    dim = len(S0)

    def simulator(normals):
        normals = np.asarray(normals)
        n_paths = normals.shape[0]
        paths = np.empty((n_paths, n_steps, dim), dtype=float)
        for i in range(n_paths):
            S = S0.copy()
            paths[i, 0, :] = S
            for j in range(1, n_steps):
                z = normals[i, j, :]
                S = S * np.exp((mu - 0.5 * sigma ** 2) * dt[j - 1] + sigma * np.sqrt(dt[j - 1]) * z)
                paths[i, j, :] = S
        return paths

    return simulator


# ---------------------------
# Przykładowe uruchomienie
# ---------------------------
if __name__ == "__main__":
    # parametry modelu
    S0 = [100.0]                # single asset
    mu = np.array([0.03])
    sigma = np.array([0.2])
    r = 0.03
    T = 1.0
    Nsteps = 100
    times = np.linspace(0.0, T, Nsteps + 1)

    # payoff: american put on single asset
    K = 100.0
    payoff = lambda j, x: max(K - float(np.sum(x)), 0.0)

    # stwórz symulator i pricer
    sim = gbm_simulator_factory(S0, mu, sigma, times)
    pricer = LSM_MARS_QMC(simulator_func=sim, payoff_func=payoff, times=times, r=r, dim=1,
                          mars_params={"min_leaf": 5, "penalty": 3.0}, rng_seed=12345)
    # ustaw parametry modelu potrzebne do simulator_one_step
    pricer.set_model_params(mu=mu, sigma=sigma)

    # uruchom (dobierz N1,N2,inner_MC w zależności od mocy obliczeniowej)
    for N1 in (1024, 2048, 4096, 8192):
        for N2 in (1024, 2048, 4096, 8192):
            res = pricer.price(N1=N1, N2=N2, max_terms=8, inner_MC=32)
            print(f"Przedział wyceny: L̂ = {res['L_hat']:.6f}, Û = {res['U_hat']:.6f}, N1 = {N1}, N2 = {N2}")
